In [1]:
from collections import Counter

import nltk
import NMT_Model
import nmt_data_utils
import nmt_model_utils

In [2]:
with open('news-commentary-v8.de-en.en',
          'r',
          encoding = 'utf-8') as f:
    en = f.readlines()
    
with open('news-commentary-v8.de-en.de',
          'r',
          encoding = 'utf-8') as f:
    de = f.readlines()

In [3]:
# first 5 sentence pairs. 
for line in zip(en[:5], de[:5]):
    print(line, '\n')

('SAN FRANCISCO – It has never been easy to have a rational conversation about the value of gold.\n', 'SAN FRANCISCO – Es war noch nie leicht, ein rationales Gespräch über den Wert von Gold zu führen.\n') 

('Lately, with gold prices up more than 300% over the last decade, it is harder than ever.\n', 'In letzter Zeit allerdings ist dies schwieriger denn je, ist doch der Goldpreis im letzten Jahrzehnt um über 300 Prozent angestiegen.\n') 

('Just last December, fellow economists Martin Feldstein and Nouriel Roubini each penned op-eds bravely questioning bullish market sentiment, sensibly pointing out gold’s risks.\n', 'Erst letzten Dezember verfassten meine Kollegen Martin Feldstein und Nouriel Roubini Kommentare, in denen sie mutig die vorherrschende optimistische Marktstimmung hinterfragten und sehr überlegt auf die Risiken des Goldes \xa0hinwiesen.\n') 

('Wouldn’t you know it?\n', 'Und es kam, wie es kommen musste.\n') 

('Since their articles appeared, the price of gold has moved u

In [4]:
# remove unnecessary new lines. 
de = [line.strip() for line in de]
en = [line.strip() for line in en]

In [5]:
# we will only use sentences of similar lengths in order to make training easier. 
len_en = [len(sent) for sent in en if 20 < len(sent) < 50]
len_dist = Counter(len_en).most_common()
len_dist

[(49, 599),
 (48, 599),
 (46, 583),
 (47, 547),
 (43, 514),
 (44, 512),
 (45, 511),
 (41, 509),
 (40, 503),
 (42, 490),
 (39, 477),
 (38, 443),
 (37, 438),
 (36, 421),
 (34, 412),
 (33, 365),
 (32, 358),
 (31, 353),
 (35, 346),
 (30, 326),
 (28, 324),
 (27, 273),
 (25, 260),
 (29, 254),
 (26, 250),
 (24, 233),
 (23, 232),
 (22, 214),
 (21, 208)]

In [6]:
# 11554 sentences that contain betwenn 20 and 50 words.
len(len_en)

11554

In [7]:
_de = []
_en = []
for sent_de, sent_en in zip(de, en):
    if 20 < len(sent_en) < 50:
        _de.append(sent_de)
        _en.append(sent_en)

In [8]:
# but we will not use all 150 000 sentences, only 5000 for the beginning.
text = _en[:2000]
language='english'
lower=True
words = []
tokenized_text = []

for line in text:
    tokenized = nltk.word_tokenize(line, language=language)
    if lower:
        tokenized = [word.lower() for word in tokenized]
    tokenized_text.append(tokenized)
    for word in tokenized:
        words.append(word)

most_common = Counter(words).most_common()
en_preprocessed = tokenized_text
en_most_common = most_common

text = _de[:2000]
language='german'
lower=True
words = []
tokenized_text = []

for line in text:
    tokenized = nltk.word_tokenize(line, language=language)
    if lower:
        tokenized = [word.lower() for word in tokenized]
    tokenized_text.append(tokenized)
    for word in tokenized:
        words.append(word)

most_common = Counter(words).most_common()
de_preprocessed = tokenized_text
de_most_common = most_common

en_preprocessed

[['wouldn', '’', 't', 'you', 'know', 'it', '?'],
 ['since',
  'then',
  ',',
  'the',
  'index',
  'has',
  'climbed',
  'above',
  '10,000',
  '.'],
 ['they',
  'departed',
  'pledging',
  'to',
  'revive',
  'europe',
  "'s",
  'growth',
  '.'],
 ['we', "'ve", 'heard', 'that', 'empty', 'promise', 'before', '.'],
 ['many', 'europeans', 'are', 'sick', 'of', 'british', 'vetoes', '.'],
 ['precedents', 'exist', 'for', 'this', '.'],
 ['nato', 'has', 'been', 'flexible', 'since', 'its', 'inception', '.'],
 ['these', 'precedents', 'can', 'be', 'extended', '.'],
 ['i', 'am', 'not', 'talking', 'about', 'a', 'two‑speed', 'europe', '.'],
 ['but', 'i', 'do', "n't", 'want', 'to', 'block', 'their', 'way', '.'],
 ['elected', 'parliaments', 'do', 'not', 'own', 'our', 'liberties', '.'],
 ['but', 'the', 'big', 'question', 'remains', ':', 'how', '?'],
 ['the', 'need', 'for', 'immediate', 'action', 'is', 'clear', '.'],
 ['the', 'new', 'year', 'is', 'looking', 'grim', '.'],
 ['france', 'is', 'flat-lining',

In [9]:
len(en_preprocessed), len(de_preprocessed)

(2000, 2000)

In [10]:
# for some of the sentences there is not german or english counterpart, i.e. only an empy array []
# therefore we will remove those sentence pairs.
en_preprocessed_clean, de_preprocessed_clean = [], []

for sent_en, sent_de in zip(en_preprocessed, de_preprocessed):
    if sent_en != [] and sent_de != []:
        en_preprocessed_clean.append(sent_en)
        de_preprocessed_clean.append(sent_de)
    else:
        continue

In [11]:
len(en_preprocessed_clean), len(de_preprocessed_clean)

(1992, 1992)

In [12]:
for e, d in zip(en_preprocessed_clean, de_preprocessed_clean[:5]):
    print('English:\n', e)
    print('German:\n', d, '\n'*3)

English:
 ['wouldn', '’', 't', 'you', 'know', 'it', '?']
German:
 ['und', 'es', 'kam', ',', 'wie', 'es', 'kommen', 'musste', '.'] 



English:
 ['since', 'then', ',', 'the', 'index', 'has', 'climbed', 'above', '10,000', '.']
German:
 ['seit', 'damals', 'ist', 'er', 'auf', 'über', '10.000', 'punkte', 'gestiegen', '.'] 



English:
 ['they', 'departed', 'pledging', 'to', 'revive', 'europe', "'s", 'growth', '.']
German:
 ['mit', 'der', 'zusicherung', ',', 'das', 'wachstum', 'in', 'europa', 'wieder', 'zu', 'beleben', ',', 'gingen', 'sie', 'auseinander', '.'] 



English:
 ['we', "'ve", 'heard', 'that', 'empty', 'promise', 'before', '.']
German:
 ['dieses', 'leere', 'versprechen', 'haben', 'wir', 'schon', 'einmal', 'gehört', '.'] 



English:
 ['many', 'europeans', 'are', 'sick', 'of', 'british', 'vetoes', '.']
German:
 ['viele', 'europäer', 'sind', 'die', 'britischen', 'vetos', 'leid', '.'] 





In [13]:
en_most_common[:15], len(en_most_common), len(de_most_common)

([('.', 1747),
  ('the', 648),
  ('is', 559),
  (',', 437),
  ('to', 297),
  ('this', 258),
  ('a', 239),
  ('but', 234),
  ('of', 213),
  ('not', 197),
  ('in', 193),
  ('are', 192),
  ('?', 184),
  ('it', 182),
  ('be', 161)],
 3174,
 7078)

In [14]:
# now we can create oyr lookup dicts for english and german, i.e. our vocab. 
# we will also include special tokens, later on used in the model. 
specials = ["<unk>", "<s>", "</s>", '<pad>']

en_word2ind, en_ind2word, en_vocab_size = nmt_data_utils.create_vocab(en_most_common, specials)
de_word2ind, de_ind2word, de_vocab_size = nmt_data_utils.create_vocab(de_most_common, specials)

In [15]:
# in order to feed the sentences to the network, we have to convert them to ints, corresponding to their indices
# in the lookup dicts. 
# we reverse the source language sentences, i.e. the english sentences as this alleviates learning for the seq2seq 
# model. Apart from this we also include EndOfSentence and StartOfSentence tags, which are needed as well. 
en_inds, en_unknowns = nmt_data_utils.convert_to_inds(en_preprocessed_clean, en_word2ind, reverse = True, eos = True)
de_inds, de_unknowns = nmt_data_utils.convert_to_inds(de_preprocessed_clean, de_word2ind, sos = True, eos = True)

In [16]:
[nmt_data_utils.convert_to_words(sentence, en_ind2word) for sentence in  en_inds[:2]]

[['?', 'it', 'know', 'you', 't', '’', 'wouldn', '</s>'],
 ['.',
  '10,000',
  'above',
  'climbed',
  'has',
  'index',
  'the',
  ',',
  'then',
  'since',
  '</s>']]

In [17]:
[nmt_data_utils.convert_to_words(sentence, de_ind2word) for sentence in  de_inds[:2]]

[['<s>',
  'und',
  'es',
  'kam',
  ',',
  'wie',
  'es',
  'kommen',
  'musste',
  '.',
  '</s>'],
 ['<s>',
  'seit',
  'damals',
  'ist',
  'er',
  'auf',
  'über',
  '10.000',
  'punkte',
  'gestiegen',
  '.',
  '</s>']]

In [23]:
# hyperparams. 
# those are probably not perfect, but work fine for now. 
num_layers_encoder = 4
num_layers_decoder = 4
rnn_size_encoder = 128
rnn_size_decoder = 128
embedding_dim = 300

batch_size = 64
epochs = 500
clip = 0
keep_probability = 0.8
learning_rate = 0.01
learning_rate_decay_steps = 1000
learning_rate_decay = 0.9

In [24]:
# create the graph and train the model. 
nmt_model_utils.reset_graph()

nmt = NMT_Model.NMT(en_word2ind,
                    en_ind2word,
                    de_word2ind,
                    de_ind2word,
                    './models/local_one/my_model',
                    'TRAIN',
                    embedding_dim = embedding_dim,
                    num_layers_encoder = num_layers_encoder,
                    num_layers_decoder = num_layers_decoder,
                    batch_size = batch_size,
                    clip = clip,
                    keep_probability = keep_probability,
                    learning_rate = learning_rate,
                    epochs = epochs,
                    rnn_size_encoder = rnn_size_encoder,
                    rnn_size_decoder = rnn_size_decoder, 
                    learning_rate_decay_steps = learning_rate_decay_steps,
                    learning_rate_decay = learning_rate_decay)

nmt.build_graph()
nmt.train(en_inds, de_inds)

Graph built.
-------------------- Epoch 0 of 500 --------------------
Iteration: 0 of 31	train_loss: 8.8653
Iteration: 2 of 31	train_loss: 8.4418
Iteration: 4 of 31	train_loss: 7.1209
Iteration: 6 of 31	train_loss: 6.8082
Iteration: 8 of 31	train_loss: 7.0854
Iteration: 10 of 31	train_loss: 7.0433
Iteration: 12 of 31	train_loss: 6.9854
Iteration: 14 of 31	train_loss: 6.8058
Iteration: 16 of 31	train_loss: 6.9491
Iteration: 18 of 31	train_loss: 7.0472
Iteration: 20 of 31	train_loss: 6.8571
Iteration: 22 of 31	train_loss: 7.0349
Iteration: 24 of 31	train_loss: 6.8777
Iteration: 26 of 31	train_loss: 6.9041
Iteration: 28 of 31	train_loss: 6.9295
Iteration: 30 of 31	train_loss: 6.7909
Iteration: 31 of 31	train_loss: 7.0253
Average Score for this Epoch: 7.150923252105713
--- new best score ---


-------------------- Epoch 1 of 500 --------------------
Iteration: 0 of 31	train_loss: 5.9994
Iteration: 2 of 31	train_loss: 6.4225
Iteration: 4 of 31	train_loss: 6.3882
Iteration: 6 of 31	train_los

Iteration: 14 of 31	train_loss: 5.4552
Iteration: 16 of 31	train_loss: 5.6768
Iteration: 18 of 31	train_loss: 5.4341
Iteration: 20 of 31	train_loss: 5.7149
Iteration: 22 of 31	train_loss: 5.8715
Iteration: 24 of 31	train_loss: 5.8117
Iteration: 26 of 31	train_loss: 5.8865
Iteration: 28 of 31	train_loss: 5.6782
Iteration: 30 of 31	train_loss: 5.8037
Iteration: 31 of 31	train_loss: 5.1783
Average Score for this Epoch: 5.606288909912109
--- new best score ---


-------------------- Epoch 11 of 500 --------------------
Iteration: 0 of 31	train_loss: 5.5212
Iteration: 2 of 31	train_loss: 5.6722
Iteration: 4 of 31	train_loss: 5.5002
Iteration: 6 of 31	train_loss: 5.4673
Iteration: 8 of 31	train_loss: 5.5200
Iteration: 10 of 31	train_loss: 5.6041
Iteration: 12 of 31	train_loss: 5.6339
Iteration: 14 of 31	train_loss: 5.3053
Iteration: 16 of 31	train_loss: 5.5354
Iteration: 18 of 31	train_loss: 5.3967
Iteration: 20 of 31	train_loss: 5.5877
Iteration: 22 of 31	train_loss: 5.7372
Iteration: 24 of

Iteration: 30 of 31	train_loss: 4.3199
Iteration: 31 of 31	train_loss: 4.0534
Average Score for this Epoch: 4.09671688079834
--- new best score ---


-------------------- Epoch 21 of 500 --------------------
Iteration: 0 of 31	train_loss: 3.8209
Iteration: 2 of 31	train_loss: 3.8293
Iteration: 4 of 31	train_loss: 3.9649
Iteration: 6 of 31	train_loss: 3.7826
Iteration: 8 of 31	train_loss: 3.9074
Iteration: 10 of 31	train_loss: 4.0884
Iteration: 12 of 31	train_loss: 3.9819
Iteration: 14 of 31	train_loss: 4.1199
Iteration: 16 of 31	train_loss: 3.9904
Iteration: 18 of 31	train_loss: 4.2361
Iteration: 20 of 31	train_loss: 3.9250
Iteration: 22 of 31	train_loss: 4.1579
Iteration: 24 of 31	train_loss: 4.1490
Iteration: 26 of 31	train_loss: 4.0524
Iteration: 28 of 31	train_loss: 3.9714
Iteration: 30 of 31	train_loss: 4.0834
Iteration: 31 of 31	train_loss: 3.8309
Average Score for this Epoch: 4.005855560302734
--- new best score ---


-------------------- Epoch 22 of 500 --------------------
Ite

Iteration: 6 of 31	train_loss: 3.4929
Iteration: 8 of 31	train_loss: 3.1774
Iteration: 10 of 31	train_loss: 3.5863
Iteration: 12 of 31	train_loss: 3.2085
Iteration: 14 of 31	train_loss: 3.3735
Iteration: 16 of 31	train_loss: 3.3693
Iteration: 18 of 31	train_loss: 3.2988
Iteration: 20 of 31	train_loss: 3.5241
Iteration: 22 of 31	train_loss: 3.1803
Iteration: 24 of 31	train_loss: 3.5062
Iteration: 26 of 31	train_loss: 3.3920
Iteration: 28 of 31	train_loss: 3.4494
Iteration: 30 of 31	train_loss: 3.5658
Iteration: 31 of 31	train_loss: 3.1174
Average Score for this Epoch: 3.370786190032959
--- new best score ---


-------------------- Epoch 32 of 500 --------------------
Iteration: 0 of 31	train_loss: 3.1669
Iteration: 2 of 31	train_loss: 3.0435
Iteration: 4 of 31	train_loss: 3.1050
Iteration: 6 of 31	train_loss: 3.2824
Iteration: 8 of 31	train_loss: 3.4980
Iteration: 10 of 31	train_loss: 3.2040
Iteration: 12 of 31	train_loss: 3.1985
Iteration: 14 of 31	train_loss: 3.2005
Iteration: 16 of 3

Iteration: 22 of 31	train_loss: 2.9159
Iteration: 24 of 31	train_loss: 3.0288
Iteration: 26 of 31	train_loss: 2.8284
Iteration: 28 of 31	train_loss: 2.9954
Iteration: 30 of 31	train_loss: 2.9909
Iteration: 31 of 31	train_loss: 2.4740
Average Score for this Epoch: 2.846193313598633
--- new best score ---


-------------------- Epoch 42 of 500 --------------------
Iteration: 0 of 31	train_loss: 2.8633
Iteration: 2 of 31	train_loss: 2.8846
Iteration: 4 of 31	train_loss: 2.8071
Iteration: 6 of 31	train_loss: 2.4295
Iteration: 8 of 31	train_loss: 2.6413
Iteration: 10 of 31	train_loss: 2.9567
Iteration: 12 of 31	train_loss: 2.8690
Iteration: 14 of 31	train_loss: 2.9055
Iteration: 16 of 31	train_loss: 2.9064
Iteration: 18 of 31	train_loss: 2.6450
Iteration: 20 of 31	train_loss: 2.9579
Iteration: 22 of 31	train_loss: 2.5343
Iteration: 24 of 31	train_loss: 2.8410
Iteration: 26 of 31	train_loss: 2.6596
Iteration: 28 of 31	train_loss: 2.8632
Iteration: 30 of 31	train_loss: 2.7782
Iteration: 31 of

...

Iteration: 26 of 31	train_loss: 0.1147
Iteration: 28 of 31	train_loss: 0.1302
Iteration: 30 of 31	train_loss: 0.1116
Iteration: 31 of 31	train_loss: 0.0883
Average Score for this Epoch: 0.11398248374462128
--- new best score ---


-------------------- Epoch 408 of 500 --------------------
Iteration: 0 of 31	train_loss: 0.1057
Iteration: 2 of 31	train_loss: 0.1202
Iteration: 4 of 31	train_loss: 0.1185
Iteration: 6 of 31	train_loss: 0.1081
Iteration: 8 of 31	train_loss: 0.0841
Iteration: 10 of 31	train_loss: 0.1238
Iteration: 12 of 31	train_loss: 0.1167
Iteration: 14 of 31	train_loss: 0.1202
Iteration: 16 of 31	train_loss: 0.1211
Iteration: 18 of 31	train_loss: 0.1076
Iteration: 20 of 31	train_loss: 0.1236
Iteration: 22 of 31	train_loss: 0.1176
Iteration: 24 of 31	train_loss: 0.1013
Iteration: 26 of 31	train_loss: 0.1198
Iteration: 28 of 31	train_loss: 0.1160
Iteration: 30 of 31	train_loss: 0.1211
Iteration: 31 of 31	train_loss: 0.1039
Average Score for this Epoch: 0.1099623441696167
---

Iteration: 6 of 31	train_loss: 0.1046
Iteration: 8 of 31	train_loss: 0.0922
Iteration: 10 of 31	train_loss: 0.0910
Iteration: 12 of 31	train_loss: 0.1000
Iteration: 14 of 31	train_loss: 0.1172
Iteration: 16 of 31	train_loss: 0.1103
Iteration: 18 of 31	train_loss: 0.1156
Iteration: 20 of 31	train_loss: 0.0923
Iteration: 22 of 31	train_loss: 0.1096
Iteration: 24 of 31	train_loss: 0.1325
Iteration: 26 of 31	train_loss: 0.0987
Iteration: 28 of 31	train_loss: 0.0938
Iteration: 30 of 31	train_loss: 0.1005
Iteration: 31 of 31	train_loss: 0.0926
Average Score for this Epoch: 0.10432592034339905
-------------------- Epoch 419 of 500 --------------------
Iteration: 0 of 31	train_loss: 0.0899
Iteration: 2 of 31	train_loss: 0.0993
Iteration: 4 of 31	train_loss: 0.0732
Iteration: 6 of 31	train_loss: 0.0820
Iteration: 8 of 31	train_loss: 0.0847
Iteration: 10 of 31	train_loss: 0.1317
Iteration: 12 of 31	train_loss: 0.1488
Iteration: 14 of 31	train_loss: 0.1110
Iteration: 16 of 31	train_loss: 0.0783
I

Iteration: 28 of 31	train_loss: 0.1146
Iteration: 30 of 31	train_loss: 0.0874
Iteration: 31 of 31	train_loss: 0.1072
Average Score for this Epoch: 0.09371429681777954
--- new best score ---


-------------------- Epoch 429 of 500 --------------------
Iteration: 0 of 31	train_loss: 0.0839
Iteration: 2 of 31	train_loss: 0.1136
Iteration: 4 of 31	train_loss: 0.0904
Iteration: 6 of 31	train_loss: 0.0865
Iteration: 8 of 31	train_loss: 0.0824
Iteration: 10 of 31	train_loss: 0.0862
Iteration: 12 of 31	train_loss: 0.0793
Iteration: 14 of 31	train_loss: 0.1022
Iteration: 16 of 31	train_loss: 0.0968
Iteration: 18 of 31	train_loss: 0.0842
Iteration: 20 of 31	train_loss: 0.0949
Iteration: 22 of 31	train_loss: 0.0985
Iteration: 24 of 31	train_loss: 0.0740
Iteration: 26 of 31	train_loss: 0.1004
Iteration: 28 of 31	train_loss: 0.0581
Iteration: 30 of 31	train_loss: 0.0973
Iteration: 31 of 31	train_loss: 0.0885
Average Score for this Epoch: 0.08893929421901703
--- new best score ---


----------------

Iteration: 8 of 31	train_loss: 0.0933
Iteration: 10 of 31	train_loss: 0.0784
Iteration: 12 of 31	train_loss: 0.1086
Iteration: 14 of 31	train_loss: 0.0832
Iteration: 16 of 31	train_loss: 0.0825
Iteration: 18 of 31	train_loss: 0.0621
Iteration: 20 of 31	train_loss: 0.0637
Iteration: 22 of 31	train_loss: 0.0685
Iteration: 24 of 31	train_loss: 0.0723
Iteration: 26 of 31	train_loss: 0.1038
Iteration: 28 of 31	train_loss: 0.0659
Iteration: 30 of 31	train_loss: 0.0607
Iteration: 31 of 31	train_loss: 0.0887
Average Score for this Epoch: 0.08064322173595428
--- new best score ---


-------------------- Epoch 440 of 500 --------------------
Iteration: 0 of 31	train_loss: 0.0861
Iteration: 2 of 31	train_loss: 0.0935
Iteration: 4 of 31	train_loss: 0.0740
Iteration: 6 of 31	train_loss: 0.0783
Iteration: 8 of 31	train_loss: 0.0727
Iteration: 10 of 31	train_loss: 0.1058
Iteration: 12 of 31	train_loss: 0.0890
Iteration: 14 of 31	train_loss: 0.0862
Iteration: 16 of 31	train_loss: 0.0930
Iteration: 18 

Iteration: 26 of 31	train_loss: 0.0579
Iteration: 28 of 31	train_loss: 0.0737
Iteration: 30 of 31	train_loss: 0.0723
Iteration: 31 of 31	train_loss: 0.0572
Average Score for this Epoch: 0.07578036189079285
-------------------- Epoch 450 of 500 --------------------
Iteration: 0 of 31	train_loss: 0.0623
Iteration: 2 of 31	train_loss: 0.0609
Iteration: 4 of 31	train_loss: 0.0868
Iteration: 6 of 31	train_loss: 0.0565
Iteration: 8 of 31	train_loss: 0.0903
Iteration: 10 of 31	train_loss: 0.0583
Iteration: 12 of 31	train_loss: 0.0606
Iteration: 14 of 31	train_loss: 0.0771
Iteration: 16 of 31	train_loss: 0.0956
Iteration: 18 of 31	train_loss: 0.0802
Iteration: 20 of 31	train_loss: 0.0655
Iteration: 22 of 31	train_loss: 0.0626
Iteration: 24 of 31	train_loss: 0.0666
Iteration: 26 of 31	train_loss: 0.0663
Iteration: 28 of 31	train_loss: 0.0792
Iteration: 30 of 31	train_loss: 0.0777
Iteration: 31 of 31	train_loss: 0.0711
Average Score for this Epoch: 0.07706300169229507
-------------------- Epoch 

In [25]:
_de_inds, _de_unknowns = nmt_data_utils.convert_to_inds(de_preprocessed_clean, de_word2ind, sos = True,  eos = True)

In [26]:
# the inference model does not necessaryly need to get input batches. we can just give it. the whole input
# data, but the the batchsize has to be specified as the lenght of the input data.
nmt_model_utils.reset_graph()

nmt = NMT_Model.NMT(en_word2ind,
                    en_ind2word,
                    de_word2ind,
                    de_ind2word,
                    './models/local_one/my_model',
                    'INFER',
                    num_layers_encoder = num_layers_encoder,
                    num_layers_decoder = num_layers_decoder,
                    batch_size = len(en_inds[:50]),
                    keep_probability = 1.0,
                    learning_rate = 0.0,
                    beam_width = 0,
                    rnn_size_encoder = rnn_size_encoder,
                    rnn_size_decoder = rnn_size_decoder)

nmt.build_graph()
preds = nmt.infer(en_inds[:50], restore_path =  './models/local_one/my_model', targets = _de_inds[:50])

Graph built.
Restore graph from  ./models/local_one/my_model
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./models/local_one/my_model


In [27]:
# show some of the created translations
# Note: the way bleu score is probably not the perfect way to do it
nmt_model_utils.sample_results(preds, en_ind2word, de_ind2word, en_word2ind, de_word2ind, _de_inds[:50], en_inds[:50])




 ----------------------------------------------------------------------------------------------------
Actual Text:
wouldn ’ t you know it ?

Actual translation:
und es kam , wie es kommen musste .

Created translation:
wo leisten dies wir wir wir kommen bei einiges vor geleistet

Bleu-score: 1.0003688322288243e-231



 ----------------------------------------------------------------------------------------------------
Actual Text:
since then , the index has climbed above 10,000 .

Actual translation:
seit damals ist er auf über 10.000 punkte gestiegen .

Created translation:
seit damals über er ungefähr buch 10.000 schon gestiegen gestiegen 0,5 nbsp

Bleu-score: 6.887578243315168e-155



 ----------------------------------------------------------------------------------------------------
Actual Text:
they departed pledging to revive europe 's growth .

Actual translation:
mit der zusicherung , das wachstum in europa wieder zu beleben , gingen sie auseinander .

Created translation:


C:\Users\ASUS\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ASUS\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ASUS\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.